In [95]:
import tensorflow_recommenders as tfrs
import tensorflow as tf
import pandas as pd
import numpy as np
from typing import Dict, Text

In [96]:
df_ratings = pd.read_csv("../../Dataset/Tourism Rating/ratings.csv")   
df_tourism = pd.read_csv("../../Dataset/Tourism/tourism_encoded.csv")


In [97]:
df_ratings.columns

Index(['Place_Id', 'Place_Name', 'User_Id', 'Place_Ratings'], dtype='object')

In [98]:
df_ratings['User_Id'] = df_ratings['User_Id'].astype('str')

In [99]:
tourism = tf.data.Dataset.from_tensor_slices(dict(df_tourism))
ratings = tf.data.Dataset.from_tensor_slices(dict(df_ratings))

In [100]:
ratings = ratings.map(lambda x:{
    "user_id":x["User_Id"],
    "place_name":x["Place_Name"]
})
tourism = tourism.map(lambda x:x["Place_Name"])

In [101]:
print(ratings)
print(tourism)

<MapDataset element_spec={'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'place_name': TensorSpec(shape=(), dtype=tf.string, name=None)}>
<MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [102]:
user_id_vocab = tf.keras.layers.StringLookup(mask_token=None)
user_id_vocab.adapt(ratings.map(lambda x: x["user_id"]))

tourism_vocab = tf.keras.layers.StringLookup(mask_token=None)
tourism_vocab.adapt(tourism)

In [103]:
print(user_id_vocab.get_vocabulary())
print(tourism_vocab.get_vocabulary())

['[UNK]', '267', '276', '40', '143', '17', '142', '54', '49', '242', '19', '184', '99', '289', '247', '227', '168', '157', '105', '91', '86', '74', '294', '179', '97', '53', '31', '28', '230', '198', '170', '159', '144', '126', '65', '38', '283', '274', '258', '206', '195', '194', '167', '146', '107', '45', '44', '33', '299', '27', '200', '187', '160', '131', '124', '116', '93', '92', '67', '58', '288', '228', '212', '208', '178', '173', '136', '94', '78', '287', '263', '262', '261', '260', '257', '250', '234', '224', '204', '182', '181', '145', '134', '84', '71', '57', '55', '297', '296', '286', '282', '277', '239', '231', '223', '201', '192', '175', '15', '149', '117', '76', '68', '64', '61', '30', '273', '26', '240', '226', '222', '216', '202', '165', '164', '139', '138', '137', '113', '108', '102', '96', '79', '63', '32', '292', '29', '285', '272', '265', '255', '248', '20', '189', '185', '18', '174', '163', '135', '133', '127', '123', '12', '118', '106', '9', '80', '5', '36', '281

In [155]:
class CollaborativeModel(tfrs.Model):
    def __init__(self):
        super().__init__()
        self.user_model = tf.keras.Sequential([
            user_id_vocab,
            tf.keras.layers.Embedding(user_id_vocab.vocabulary_size(), 16)
        ])
        self.tourism_model = tf.keras.Sequential([
            tourism_vocab,
            tf.keras.layers.Embedding(tourism_vocab.vocabulary_size(), 16)
        ])
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=tourism.batch(128).map(self.tourism_model)
            )
        )
    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        tourism_embeddings = self.tourism_model(features["place_name"])
        return self.task(user_embeddings, tourism_embeddings)

In [156]:
model = CollaborativeModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad())
model.fit(ratings.batch(32), epochs=10)

Epoch 1/10


313/313 [==============================] - 33s 100ms/step - factorized_top_k/top_1_categorical_accuracy: 4.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0080 - factorized_top_k/top_10_categorical_accuracy: 0.0183 - factorized_top_k/top_50_categorical_accuracy: 0.1109 - factorized_top_k/top_100_categorical_accuracy: 0.2263 - loss: 110.4808 - regularization_loss: 0.0000e+00 - total_loss: 110.4808
Epoch 2/10
313/313 [==============================] - 32s 101ms/step - factorized_top_k/top_1_categorical_accuracy: 3.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0083 - factorized_top_k/top_10_categorical_accuracy: 0.0194 - factorized_top_k/top_50_categorical_accuracy: 0.1131 - factorized_top_k/top_100_categorical_accuracy: 0.2291 - loss: 110.4797 - regularization_loss: 0.0000e+00 - total_loss: 110.4797
Epoch 3/10
313/313 [==============================] - 33s 106ms/step - factorized_top_k/top_1_categorical_accuracy: 3.0000e-04 - factorized_top_k/top_5_categorical_accur

In [159]:
index = tfrs.layers.factorized_top_k.BruteForce (model.user_model)
no_user = 1
index.index_from_dataset(tourism.batch(100).map(lambda place: (place,model.tourism_model(place))))
_, titles = index(np.array([str(no_user)]))
print(f"Recommendations for user {no_user}: {titles[0,:10]}")

Recommendations for user 1: [b'Surabaya Museum (Gedung Siola)' b'Pasar Beringharjo'
 b'Grojogan Watu Purbo Bangunrejo' b'Galeri Indonesia Kaya'
 b'Situ Cileunca' b'Kampung Wisata Rejowinangun' b'Pantai Sadranan'
 b'Umbul Sidomukti' b'Pelabuhan Marina' b'Sumur Gumuling']


In [160]:
model.save_weights("./model2/model_weights")